In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp

In [ ]:
from GraphST import GraphST

In [ ]:
# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
#os.environ['R_HOME'] = '/scbio4/tools/R/R-4.0.3_openblas/R-4.0.3'

In [ ]:
# read data
adata=sc.read_h5ad("./data/MISAR_seq/E15_5-S1_expr_gt.h5ad")
adata.var_names_make_unique()

print(adata)

In [ ]:
# define model
model = GraphST.GraphST(adata, device=device)

# train model
adata = model.train()

In [ ]:
# set radius to specify the number of neighbors considered during refinement
radius = 10

tool = 'mclust' # mclust, leiden, and louvain

# clustering
from GraphST.utils import clustering

if tool == 'mclust':
   clustering(adata, 10, radius=radius, method=tool, refinement=False) # For DLPFC dataset, we use optional refinement step.
elif tool in ['leiden', 'louvain']:
   clustering(adata, 10, radius=radius, method=tool, start=0.1, end=2.0, increment=0.01, refinement=False)

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='domain', title='GraphST', s=150, show=False)